# Jupyter Lectures, First Year Project 2021

<font color="red">Instructor's version</font>

## Project 1, ITU Copenhagen

**Instructor: Michael Szell**

Course page: https://learnit.itu.dk/local/coursebase/view.php?ciid=590

This notebook wrangles and explores the data set from the project.

Contact: Michael Szell (misz@itu.dk)  
Created: 2021-01-11  
Last modified: 2021-01-26

<hr>

# Lecture 1: First data exploration

### Imports

In [ ]:
# First version
import numpy as np


# Add later as needed
import matplotlib
import matplotlib.pyplot as plt
import csv

import pprint
pp = pprint.PrettyPrinter(indent=4)

import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
# Add later as needed
%run -i ../scripts/functions.py

### Constants

Constants are written all caps: https://www.python.org/dev/peps/pep-0008/#constants

In [ ]:
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"

FILENAME = {}
FILENAME["accidents"] = "Road Safety Data - Accidents 2019.csv"
FILENAME["casualties"] = "Road Safety Data - Casualties 2019.csv"
FILENAME["vehicles"] = "Road Safety Data- Vehicles 2019.csv" # Note the inconsistent file naming (no space before "-" here)

# Add later, lec 2
TABLENAMES = ["accidents", "casualties", "vehicles"]

### Load raw data

The data were downloaded from here on Jan 4th: https://data.gov.uk/dataset/road-accidents-safety-data  
That page was updated afterwards (Jan 8th), so local and online data may be inconsistent.

We first explore one data table, the accidents.

In [ ]:
# First version, just using the accident table
dataraw_acc = np.genfromtxt(PATH["data_raw"]+FILENAME["accidents"], delimiter=',', dtype=None, names=True, encoding='utf8')

# # Add later, end of lec 1
# dataraw = {}
# dataraw["accidents"] = np.genfromtxt(PATH["data_raw"]+FILENAME["accidents"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
# dataraw["vehicles"] = np.genfromtxt(PATH["data_raw"]+FILENAME["vehicles"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
# dataraw["casualties"] = np.genfromtxt(PATH["data_raw"]+FILENAME["casualties"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

# Add later, in lec 2
dataraw = {}
for tablename in TABLENAMES:
    dataraw[tablename] = np.genfromtxt(PATH["data_raw"]+FILENAME[tablename], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

In [ ]:
# Add later, in lec 2
headerraw = {}
for tablename in TABLENAMES:
    headerraw[tablename] = list(dataraw[tablename].dtype.names)

It is always good to start with a "sneak preview":

In [ ]:
dataraw_acc[:5]

Reminder and documentation on structured arrays:  
https://numpy.org/devdocs/user/basics.rec.html

#### Insight: Mixed variable types

Accidents have mixed data types, including strings, floats, integers. Categorical variables are encoded as integers. The meaning of these categories can be looked up in `../references/variable lookup.xls`

Number of records

In [ ]:
dataraw_acc.shape

Number of fields

In [ ]:
len(dataraw_acc.dtype)

In [ ]:
dataraw_acc.dtype

**"Data in the wild" puzzle: Why is the first field "\ufeffAccident_Index" and not "Accident_Index"?**

Fields

In [ ]:
dataraw_acc.dtype.names

Solution: utf8 was the wrong encoding! The correct one seems to be utf-8-sig.

https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string/17912811#17912811

<font color="red">Instructor: Go back to top and fix. Also make dict of tables. Explain code refactoring:</font>  
https://en.wikipedia.org/wiki/Code_refactoring

Homework: Explore the other two tables the same way.

<hr>

# Lecture 2: Command line wrangling and dealing with missing data

A faster way of getting basic insights into a new data set than by using numpy is by using command line tools.

Let's get a first overview using `head`. There are 3 data tables: Accidents, Casualties, and Vehicles.

In [ ]:
!head -n 6 "../data/raw/Road Safety Data - Accidents 2019.csv" 
!head -n 6 "../data/raw/Road Safety Data - Casualties 2019.csv"
!head -n 6 "../data/raw/Road Safety Data- Vehicles 2019.csv" 

### General insights

#### Link between data tables

Records between data tables are linked through their `Accident_Index`.

Looking at the first Accident_Index 2019010128300, we can see there seems to be a one-to-many relation between accident->casualty and accident->vehicle, meaning there can be multiple casualties and vehicles involved in one accident (makes sense).

https://en.wikipedia.org/wiki/One-to-many_(data_model)

#### Dimensions

Number of records

https://en.wikipedia.org/wiki/Wc_(Unix)

In [ ]:
!wc -l "../data/raw/Road Safety Data - Accidents 2019.csv" 
!wc -l "../data/raw/Road Safety Data - Casualties 2019.csv" 
!wc -l "../data/raw/Road Safety Data- Vehicles 2019.csv"

Number of fields (in first line)

https://www.geeksforgeeks.org/awk-command-unixlinux-examples/

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | awk -F ',' '{print NF}'

See and count all fields

https://en.wikipedia.org/wiki/Tr_(Unix)
https://en.wikipedia.org/wiki/Nl_(Unix)

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | tr ',' '\n' | nl

### Sanity checks

Has each record the same number of fields?

https://shapeshed.com/unix-uniq/  
https://www.putorius.net/uniq-command-linux.html

In [ ]:
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Accidents 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Casualties 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data- Vehicles 2019.csv" | sort | uniq -d

How many duplicate lines are there? (If more than 0, there could be a problem)

In [ ]:
!sort "../data/raw/Road Safety Data - Accidents 2019.csv" | uniq -d  | wc -l
!sort "../data/raw/Road Safety Data - Casualties 2019.csv" | uniq -d | wc -l
!sort "../data/raw/Road Safety Data- Vehicles 2019.csv" | uniq -d  | wc -l

More advanced stuff with `awk`: https://datafix.com.au/BASHing/2020-05-20.html

## Dealing with missing data

Using a masked array:  
https://numpy.org/devdocs/reference/maskedarray.baseclass.html#numpy.ma.MaskedArray

<font color="red">Instructor: We want a nicer way to loop through table names. Let's go to beginning and refactor with TABLENAMES.</font>

In [ ]:
dataraw_masked = {}
for tablename in TABLENAMES:
    dataraw_masked[tablename] = np.genfromtxt(PATH["data_raw"]+FILENAME[tablename], delimiter=',', dtype=None, names=True, encoding='utf-8-sig', usemask=True)

In [ ]:
dataraw_masked["accidents"][:5]

In [ ]:
dataraw_masked["accidents"].mask[:5]

The first 5 rows seem complete. What about the rest?

In [ ]:
np.count_nonzero(dataraw_masked["accidents"].mask)

Oh oh, values are missing in 5776 rows! In which rows?

In [ ]:
rows_incomplete = np.where(dataraw_masked["accidents"].mask)[0]
print(rows_incomplete)

How many values in total?  
Which fields are missing?

In [ ]:
missingpositions = {}
missingvalues = 0 # Add later
missingconfigurations = set() # Add later
for rowpos in rows_incomplete:
    missingpositions_thisrow = list(np.where(list(dataraw_masked["accidents"].mask[rowpos]))[0])
    missingpositions[rowpos] = missingpositions_thisrow
    missingvalues += len(missingpositions_thisrow) # Add later
    missingconfigurations.add((tuple(missingpositions_thisrow))) # Add later

missingfieldnames = [np.array(headerraw["accidents"])[c] for c in [list(b) for b in missingconfigurations]] # Add later

In [ ]:
pp.pprint(missingpositions) # Don't do this is you have more than a few 1000 rows or Jupyter might crash.

Summary of missing values:

In [ ]:
print("Incomplete rows: " + str(np.count_nonzero(dataraw_masked["accidents"].mask)))
print("Missing values: " + str(missingvalues))
print("\nMissing field configurations: " + str(missingconfigurations))
print("Missing field configurations (names): ")
for i in missingfieldnames:
    print(i)

# Lecture 3: Distributions, Regression, Connecting tables

In [90]:
# To do

# Lecture 4: Spatial filtering

In [ ]:
cityname = "Bristol"
cityid = "bristol"

## Filtering with external table

In [ ]:
lsoa = np.genfromtxt(PATH["data_external"] + "Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2.csv", delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

In [ ]:
lsoa

Let's select all rows for the city name:

In [ ]:
np.count_nonzero(lsoa['LSOA11NM'] == cityname)

**Oops. We did not find a single row. Why?**

https://stackoverflow.com/questions/38974168/finding-entries-containing-a-substring-in-a-numpy-array

In [ ]:
city_LSOA11CD_rows = np.flatnonzero(np.core.defchararray.find(lsoa['LSOA11NM'],cityname)!=-1)
city_LSOA11CD_list = lsoa['LSOA11CD'][city_LSOA11CD_rows] # this is a list
city_LSOA11CD = {k: 0 for k in city_LSOA11CD_list.tolist()} # Turn into dict keys. 
# Alternative: city_LSOA11CD = dict.fromkeys(city_LSOA11CD , 0)
city_LSOA11CD

We want to use this dict of LSOA11 codes to restrict our accident data set.

**Question: Why use a dict, and not a list?**

In [ ]:
%timeit 'E01033370' in city_LSOA11CD

In [ ]:
%timeit 'E01033370' in city_LSOA11CD_list

Because we will do some quite restricting of data sets, lets write a function for that, in `functions.py`

In [ ]:
# Add to beginning of notebook
%run -i ../scripts/functions.py

In [ ]:
city_accidentindices = restrict_dataset(dataraw["accidents"], city_LSOA11CD, "LSOA_of_Accident_Location")["Accident_Index"]
city_accidentindices = {k: 0 for k in city_accidentindices.tolist()} # Turn into dict keys
city_accidentindices

Instead of turning a list into a dict before calling the function, lets accept both. We extend our function:
```python
if type(indices) == "list": 
    indices = {k: 0 for k in indices}
```

Filter

In [ ]:
datacity = {}
for tablename in TABLENAMES:
    datacity[tablename] = restrict_dataset(dataraw[tablename], city_accidentindices)

Export

In [ ]:
for tablename in TABLENAMES:
    with open(PATH["data_interim"] + tablename + "_" + cityid + ".csv", "w") as f:
        w = csv.writer(f)
        w.writerow(dataraw[tablename].dtype.names)
        w.writerows(datacity[tablename])

## Spatial filtering with shapely

<font color="red">Instructor only: Run one time to get JSON</font>

In [ ]:
import json
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

In [ ]:
with open(PATH["data_external"] + "lad.json") as f:
    jsn = json.load(f)
cities = {"Birmingham": "Birmingham", "Leeds": "Leeds", "Sheffield": "Sheffield", "Bradford": "Bradford", "Liverpool": "Liverpool", "Manchester": "Manchester", "Bristol, City of": "Bristol"}

cities_json = {}
for i in range(len(jsn['features'])):
    if jsn['features'][i]["properties"]["LAD13NM"] in cities:
        cities_json[cities[jsn['features'][i]["properties"]["LAD13NM"]]] = jsn['features'][i]
        print(cities[jsn['features'][i]["properties"]["LAD13NM"]])
with open(PATH["data_processed"] + "citieslad.json", "w") as f:
    json.dump(cities_json, f)

<font color="red">Instructor only END</font>

#### For lecture

In [ ]:
# Add to beginning imports
import json
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

In [ ]:
with open(PATH["data_processed"] + "citieslad.json") as f:
    cities_json = json.load(f)

In [ ]:
cities_json

In [ ]:
city_boundary = shapely.geometry.shape(cities_json[cityname]["geometry"])
type(city_boundary)

Jupyter visualizes shapely objects!

In [ ]:
city_boundary

In [ ]:
city_boundary.geom_type

Let's get all accident coordinates (from the whole UK)

In [ ]:
lon_list = dataraw["accidents"]["Longitude"]
lat_list = dataraw["accidents"]["Latitude"]

`contains()` and `within()` check for point inclusion:

In [ ]:
print(city_boundary.contains(Point(lon_list[0], lat_list[0])))
print(city_boundary.contains(Point(lon_list[102135], lat_list[102135])))

In [ ]:
city_acc_rowindices = []
for i in range(len(lon_list)):
    if Point(lon_list[i], lat_list[i]).within(city_boundary):
        city_acc_rowindices.append(i)

In [ ]:
len(city_acc_rowindices)

In [ ]:
datacity = {}
datacity["accidents"] = dataraw["accidents"][city_acc_rowindices]
datacity["accidents"]

In [ ]:
with open(PATH["data_interim"] + "accidents_" + cityid + "lad.csv", "w") as f:
    w = csv.writer(f)
    w.writerow(data_acc.dtype.names)
    w.writerows(city_data_acc)

Limit casualties and vehicles to these AccidentIndices

In [ ]:
len(datacity["accidents"]["Accident_Index"])

In [ ]:
city_accidentindices = {}
for i in range(len(datacity["accidents"]["Accident_Index"])):
    city_accidentindices[datacity["accidents"]["Accident_Index"][i]] = 0
city_accidentindices

Filter vehicles and make an interim data set

In [ ]:
datacity["vehicles"] = restrict_dataset(dataraw["vehicles"], city_accidentindices)
datacity["vehicles"]

In [ ]:
len(datacity["vehicles"])

In [ ]:
with open(PATH["data_interim"] + "vehicles_" + cityid + "lad.csv", "w") as f:
    w = csv.writer(f)
    w.writerow(dataraw["vehicles"].dtype.names)
    w.writerows(datacity["vehicles"])

Filter casualties and make an interim data set

In [ ]:
datacity["casualties"] = restrict_dataset(dataraw["casualties"], city_accidentindices)
datacity["casualties"]

In [ ]:
len(datacity["casualties"])

In [ ]:
with open(PATH["data_interim"] + "casualties_" + cityid + "lad.csv", "w") as f:
    w = csv.writer(f)
    w.writerow(dataraw["casualties"].dtype.names)
    w.writerows(datacity["casualties"])

In [ ]:
!diff ../data/interim/accidents_bristol.csv ../data/interim/accidents_bristol_lad.csv

What does it mean?  
See: https://www.computerhope.com/unix/udiff.htm

# Lecture 5: Visualizing spatial data

In [ ]:
# Add these to the imports in the beginning
import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

https://alysivji.github.io/getting-started-with-folium.html

In [ ]:
latlons = np.vstack( (datacity["accidents"]['Latitude'], datacity["accidents"]['Longitude'])).T
centroid = list(MultiPoint(latlons).centroid.coords)[0]
m1 = folium.Map(centroid, zoom_start=11)
for row in datacity["accidents"]:
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius = 5,
                        popup = row['Accident_Index'] + "\n" + row["Date"] + ", " + row["Time"],
                        fill_color = "#3db7e4",
                       ).add_to(m1)

HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m1))
folium.LayerControl().add_to(m1)
m1

The heatmap is built with KDE:  
https://en.wikipedia.org/wiki/Kernel_density_estimation

We can also add automatic clusters:

In [ ]:
latlons = np.vstack( (datacity["accidents"]['Latitude'], datacity["accidents"]['Longitude'])).T
centroid = list(MultiPoint(latlons).centroid.coords)[0]
m2 = folium.Map(centroid, zoom_start=11)
marker_cluster = MarkerCluster().add_to(folium.FeatureGroup(name='Clusters').add_to(m2))
for row in datacity["accidents"]:
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius = 5,
                        popup = row['Accident_Index'] + "\n" + row["Date"] + ", " + row["Time"],
                        fill_color = "#3db7e4",
                       ).add_to(marker_cluster)

HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m2))
folium.LayerControl().add_to(m2)
m2